In [ ]:
# Some MQTT based main files for the esp8266 sensors in DoESLiverpool
# For cheaper deployment

# RST         |  GPIO1   TX
# A0          |  GPIO3   RX
# D0  GPIO16  |  GPIO5   D1  SCL
# D5  GPIO14  |  GPIO4   D2  SDA
# D6  GPIO12  |  GPIO0   D3
# D7  GPIO13  |  GPIO2   D4
# D8  GPIO15  |          GND
# 3V3         |          5V

In [13]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
# Code below is to run just the SI7021 humidity temp sensor
# (and then the dallas temperature string)

In [14]:
%sendtofile main.py

import time, network, machine

boardname = "esp8266B"
mqttipnumber = "10.0.30.130"  # not able to make mDNS to work
pled = machine.Pin(2, machine.Pin.OUT)

si = network.WLAN()
si.active(True)
si.connect("DoESLiverpool", "decafbad00")
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(boardname, mqttipnumber)
client.connect()
client.publish(boardname+"/ip", ipnumber, retain=True, qos=1)

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

# dallas string sensor code; G=Orange, D8=Yellow, 3V3=Blue
import onewire, ds18x20
pindallas = machine.Pin(13, machine.Pin.OUT, machine.Pin.PULL_UP)
dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
dallasscanned = dallasobj.scan()
if dallasscanned:
    sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
    dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
    print("DS18B20 n=%d"%len(dallasscanned))
    
# SI7021 humidity sensor code
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
bsi7021init = False
def SI7021humiditytemp():
    global bsi7021init
    if not bsi7021init:
        i2c.writeto(0x40, b'\xFE')
        time.sleep_ms(20)
        bsi7021init = True
    i2c.writeto(0x40, b'\xE5')
    time.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rh = (bh[0]<<8)+(bh[1]&0xFC)
    rt = (bt[0]<<8)+(bt[1]&0xFC)
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

topichumid = boardname+"/SI7021/humid"
topictemp = boardname+"/SI7021/temp"
topicds = [ boardname+"/DS18B20/"+str(i)  for i in range(len(dallasscanned)) ]

while True:
    pled.value(0)
    try:
        h, t = SI7021humiditytemp()
        print(h, t)
        client.publish(topichumid, "%f"%h)
        client.publish(topictemp, "%f"%t)
    except OSError as e:
        print(e)
    pled.value(1)
    
    dallasobj.convert_temp()
    time.sleep_ms(800)
    for i, d in enumerate(dallasscanned):
        try:
            t = dallasobj.read_temp(d)
            print("%d %.2f"%(i, t))
            client.publish(topicds[i], "%f"%t)
        except Exception:
            pass
    time.sleep_ms(200)


Sent 78 lines (2331 bytes) to main.py.
